In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import sys

sys.path.insert(0,'/content/drive/My Drive/ml-trde-notebooks')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ***** WARNING : Install deps - This will BUILD TALib and takes a while!
%run -i '/content/drive/My Drive/ml-trde-notebooks/installs.ipynb'

In [ ]:
# Executes this notebook in our space, making all of its functions/globals available
%run -i '/content/drive/My Drive/ml-trde-notebooks/common.ipynb'

Modified
1674642396
1674642396.0
1674642396.0


In [ ]:
load_all_models()

loading: lstm_cv
loading: lstm_coins_cv
loading: lstm_att_cv
loading: lstm_att_ohlcv
loading: lstm_cv_rvi
loading: lstm_cv_vwap
loading: lstm_ohlc
loading: svm_cv
loading: svm_cv_vwap
loading: lstm_xgb_cols
models loaded


In [ ]:
product = "BTC-USDT"
df_raw = sort_date(get_coin_data_frames(360, product))
[scaled_features, X, y, normal_features] = convert_to_training_dataset(df_raw)
normal_features

,Open,High,Low,Close,Volume,CDL_ENGULFING,CDL_HARAMI,CDL_HARAMICROSS,CDL_PIERCING,CDL_DARKCLOUDCOVER,...,BBB_5_2.0,BBP_5_2.0,OBV,AD,MFI_14,WILLR_14,RVI,VWAP,VWAPD,Target
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-31,37882.500000,38739.000000,36626.700000,38467.300000,8856.551803,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,8856.551803,6578.158464,0.000000,0.000000,0.000000,37944.333333,522.966667,38693.400000
2022-02-01,38467.400000,39286.000000,38021.200000,38693.400000,8866.025687,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,17722.577490,7136.140473,0.000000,0.000000,0.000000,38666.866667,26.533333,36883.900000
2022-02-02,38693.300000,38860.000000,36576.000000,36883.900000,7983.608385,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,9738.969104,1305.030355,0.000000,0.000000,0.000000,37439.966667,-556.066667,37320.900000
2022-02-03,36889.700000,37381.900000,36249.100000,37320.900000,7376.601938,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,17115.571042,7887.188934,0.000000,0.000000,0.000000,36983.966667,336.933333,41579.700000
2022-02-04,37320.900000,41756.400000,37032.000000,41579.700000,11180.303485,0.000000,0.000000,0.000000,0.000000,0.000000,...,17.023293,0.955260,28295.874528,18231.170505,0.000000,0.000000,0.000000,40122.700000,1457.000000,41386.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-21,22668.800000,23369.900000,22417.000000,22780.900000,10272.731368,0.000000,0.000000,0.000000,0.000000,0.000000,...,16.228914,0.816989,-225062.126626,-11453.620177,90.437899,-9.121318,70.451937,22855.933333,-75.033333,22707.900000
2023-01-22,22780.900000,23071.800000,22295.000000,22707.900000,8708.520715,0.000000,0.000000,0.000000,0.000000,0.000000,...,16.610080,0.698940,-233770.647342,-10904.292789,82.439860,-10.569002,57.954822,22691.566667,16.333333,22913.500000
2023-01-23,22707.900000,23175.000000,22504.100000,22913.500000,8095.727633,0.000000,0.000000,0.000000,0.000000,0.000000,...,12.177450,0.677427,-225674.919709,-9119.588485,83.096930,-7.334555,64.680377,22864.200000,49.300000,22629.000000


In [ ]:
!pip install pyalgotrade

In [ ]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import quandlfeed
from pyalgotrade.technical import ma


class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, smaPeriod):
        super(MyStrategy, self).__init__(feed, 1000)
        self.__position = None
        self.__instrument = instrument
        # We'll use adjusted close values instead of regular close values.
        self.setUseAdjustedValues(True)
        self.__sma = ma.SMA(feed[instrument].getPriceDataSeries(), smaPeriod)

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at $%.2f" % (execInfo.getPrice()))

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at $%.2f" % (execInfo.getPrice()))
        self.__position = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__sma[-1] is None:
            return

        bar = bars[self.__instrument]
        # If a position was not opened, check if we should enter a long position.
        if self.__position is None:
            if bar.getPrice() > self.__sma[-1]:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, 10, True)
        # Check if we have to exit the position.
        elif bar.getPrice() < self.__sma[-1] and not self.__position.exitActive():
            self.__position.exitMarket()


def run_strategy(smaPeriod):
    # Load the bar feed from the CSV file
    feed = quandlfeed.Feed()
    feed.addBarsFromCSV("orcl", "WIKI-ORCL-2000-quandl.csv")

    # Evaluate the strategy with the feed.
    myStrategy = MyStrategy(feed, "orcl", smaPeriod)
    myStrategy.run()
    print("Final portfolio value: $%.2f" % myStrategy.getBroker().getEquity())

run_strategy(15)